In [1]:
import numpy as np
import pandas as pd
import gzip
import os
import zipfile
import openrouteservice
client = openrouteservice.Client(key='5b3ce3597851110001cf624802e069d6633748a5ae4e9842334f1dc2') # Specify your personal API key
from openrouteservice import client
from openrouteservice.exceptions import ApiError
import openrouteservice
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely import geometry
import plotly.express as px
import time
import geopy.distance
import plotly.graph_objects as go

In [2]:
# Your directory path
directory_path = r"C:\Users\leonw\OneDrive - KU Leuven\Datathon\Data2024"

# Initialize an empty dictionary to hold your data
data = {}

# List all files and read relevant ones into the dictionary
for filename in os.listdir(directory_path):
    if filename.endswith(".parquet") or filename.endswith(".parquet.gzip"):
        # Construct the full file path
        file_path = os.path.join(directory_path, filename)
        # Remove the file extension and replace underscores with spaces for the key
        key_name = filename.replace('.parquet.gzip', '').replace('.parquet', '').replace('_', ' ')
        # Read the file into a DataFrame
        data[key_name] = pd.read_parquet(file_path)

In [3]:
aed_loc = data['aed locations']
amb_loc = data['ambulance locations']
cad9_loc = data['cad9']
intvn_bxl1 = data['interventions bxl']
intvn_bxl2 = data['interventions bxl2']
intvn1 = data['interventions1']
intvn2 = data['interventions2']
intvn3 = data['interventions3']
mug_loc = data['mug locations']
pit_loc = data['pit locations']

In [4]:
df = pd.DataFrame(columns = ['Name', 'longitude', 'latitude'])

In [5]:
amb_loc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279 entries, 0 to 278
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   base                       279 non-null    object 
 1   medical_resource           279 non-null    object 
 2   province                   266 non-null    object 
 3   region                     279 non-null    object 
 4   departure_location         279 non-null    object 
 5   departure_location_number  279 non-null    int64  
 6   latitude                   279 non-null    float64
 7   longitude                  279 non-null    float64
 8   occasional_permanence      279 non-null    object 
dtypes: float64(2), int64(1), object(6)
memory usage: 19.7+ KB


In [6]:
pd.unique(amb_loc['province'])

array(['Antwerpen', None, 'Waals-Brabant', 'Luik', 'Henegouwen',
       'Limburg', 'Namen', 'Oost-Vlaanderen', 'Vlaams-Brabant',
       'West-Vlaanderen', 'Luxemburg'], dtype=object)

In [7]:
Ambulances = amb_loc[amb_loc['province']=='Vlaams-Brabant']

In [8]:
# Only keep rows that contain 30 as all postla code of Leuven start with 30
Ambulances = Ambulances[Ambulances['departure_location'].str.contains('30')]

In [9]:
Ambulances = Ambulances.reset_index(drop = True)

In [10]:
df['Name'] = Ambulances['departure_location_number']
df['longitude'] = Ambulances['longitude']
df['latitude'] = Ambulances['latitude']

In [11]:
pd.unique(pit_loc['province'])

array(['Antwerpen/Anvers', 'Oost-Vlaanderen/Flandre orientale',
       'Vlaams Brabant/Brabant flamand',
       'West-Vlaanderen/Flandre occidentale', 'Limburg/Limbourg', None,
       'Henegouwen/Hainaut', 'Namen/Namur', 'Luik/Liège',
       'Luxemburg/Luxembourg'], dtype=object)

In [12]:
Pit = pit_loc[pit_loc['province']=='Vlaams Brabant/Brabant flamand']
Pit = Pit.reset_index()

In [13]:
Pit
# Only the second one is in Leuven

,index,unit,campus,province,region,unit_id,ambucode,ambusitecode,link
0,6,"210001: PIT ASSE, OLV",Ziekenhuis/Hôpital: 126 - ONZE-LIEVE-VROUW ZIE...,Vlaams Brabant/Brabant flamand,Vlaanderen/Flandre,PVASSE01A,480,48001,None
1,11,"501001: PIT Leuven, Gasthuisberg",Ziekenhuis/Hôpital: 322 - UZ LEUVEN --- Campus...,Vlaams Brabant/Brabant flamand,Vlaanderen/Flandre,PVLEUV01A,288,28801,None


In [14]:
Pit = Pit[Pit['ambucode']==288]

In [15]:
Pit = Pit.reset_index(drop = True)

In [16]:
# Getting the coordinates mannualy
Pit['campus'][0]

'Ziekenhuis/Hôpital: 322 - UZ LEUVEN --- Campus/Site: 4230 - GASTHUISBERG'

In [17]:
df_2 = pd.DataFrame()
df_2['Name'] = Pit['unit_id']
df_2['longitude'] = 4.671651874790573
df_2['latitude'] = 50.879170226125666

In [18]:
pd.unique(mug_loc['province'])

array(['Antwerpen', 'Oost-Vlaanderen', 'West-Vlaanderen', 'Limburg',
       'Vlaams-Brabant', 'Brussels Hoofdstedelijk Gewest',
       'Waals-Brabant', 'Henegouwen', 'Namen', 'Luik', 'Lüttich',
       'Luxemburg'], dtype=object)

In [37]:
Mug = mug_loc[mug_loc['province']=='Vlaams-Brabant']

In [20]:
# Only Leuven postal code starts with 3000

In [21]:
Mug = Mug[Mug['postal_code']==3000]

In [22]:
Mug = Mug.reset_index(drop = True)

In [41]:
Mug.head(10)

,hospital_id,mug_id,campus_id,name_hospital,name_campus,address_campus,postal_code,municipality,region,province
44,106,500000,1340,ALGEMEEN ZIEKENHUIS SINT-MARIA,None,ZIEKENHUISLAAN 100,1500,HALLE,V,Vlaams-Brabant
45,322,501000,4230,UZ LEUVEN,GASTHUISBERG,HERESTRAAT 49,3000,LEUVEN,V,Vlaams-Brabant
46,204,502000,1460,AZ JAN PORTAELS,None,GENDARMERIESTRAAT 65,1800,VILVOORDE,V,Vlaams-Brabant
47,109,503000,1370,RZ HEILIG HART TIENEN,MARIENDAL,KLINIEKSTRAAT 45,3300,TIENEN,V,Vlaams-Brabant
48,712,504000,1320,AZ DIEST,None,STATIESTRAAT 65,3290,DIEST,V,Vlaams-Brabant


In [23]:
df_3 = pd.DataFrame()
df_3['Name'] = Mug['hospital_id']
df_3['longitude'] = 4.669060381545932
df_3['latitude'] = 50.87843614224022

In [24]:
# Combine all vectors
df_vectors = pd.concat([df, df_2, df_3], ignore_index = True)

In [25]:
df_vectors['coordinates'] =  list(zip(df_vectors.loc[:,'longitude'], df_vectors.loc[:,'latitude']))

In [26]:
Vectors = df_vectors[['longitude', 'latitude', 'coordinates']]

In [35]:
Vectors

,longitude,latitude,coordinates
0,4.724344,50.869655,"(4.7243436, 50.8696549)"
1,4.695398,50.878535,"(4.6953981, 50.8785355)"
2,4.098320,50.868004,"(4.0983199, 50.8680038)"
3,4.472760,50.879821,"(4.4727595, 50.8798215)"
4,4.665862,50.877460,"(4.6658621, 50.8774603)"
5,4.527038,50.777013,"(4.5270384, 50.7770126)"
6,4.937178,50.823648,"(4.9371776, 50.8236483)"
7,4.500188,50.875499,"(4.500188, 50.8754995)"
8,4.699833,50.872354,"(4.6998331, 50.8723536)"
9,4.673199,50.878079,"(4.6731993, 50.8780791)"


In [27]:
responder_full = pd.read_csv("C:/Users/leonw/OneDrive - KU Leuven/2nd Semester/Modern Data Analytics/Assignment/1%_FRs.csv")
responder_full.rename(columns = {'x':'longitude', 'y':'latitude'}, inplace = True)
responder_full['coordinates'] = list(zip(responder_full['longitude'], responder_full['latitude']))

In [28]:
aed = pd.read_csv("C:/Users/leonw/Downloads/filtered_AED_loc.csv")
aed_location = aed[['longitude', 'latitude']]
aed_location = aed_location.reset_index(drop = True)
aed_location['coordinates'] = list(zip(aed_location['longitude'], aed_location['latitude']))

In [29]:
# Generate 100 random lat and long
N = 4
lon = np.random.uniform(4.697659,4.709659,N)
lat = np.random.uniform(50.868214,50.880214,N)

# Create dataframe from the numpy arrays
Patient = pd.DataFrame({'longitude':lon, 'latitude':lat})
Patient['coordinates'] = list(zip(Patient['longitude'], Patient['latitude']))

In [30]:
import openrouteservice
from Simulation_Class import simulation

In [33]:
test = simulation()

In [34]:
df_final = pd.DataFrame(columns = ['Patient_loc', 'Responder_lon', 'Responder_lat', 
        'duration_Responder', 'AED_lon', 'AED_lat','duration_AED',
        'Vector_lon', 'Vector_lat', 'duration_Vector'])
for i in range(len(Patient)):
    df = test.fastest_time(Patient.loc[i], 
                           responder_full, aed_location, Vectors)
    df_final = pd.concat([df_final,df], ignore_index=True)

C:\Users\leonw\anaconda3\Lib\site-packages\openrouteservice\client.py:195: UserWarning: Server down.
Retrying for the 1st time.
  warnings.warn('Server down.\nRetrying for the {0}{1} time.'.format(retry_counter + 1,


KeyboardInterrupt: 